## gathering bus details ##

In [1]:
# importing required packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import  expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


In [32]:
# importing the route details csv file
df = pd.read_csv('bus_names_and_routes.csv')
df

,Unnamed: 0,routes,links,state
0,0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC
1,1,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...,APSRTC
2,2,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC
3,3,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...,APSRTC
4,4,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...,APSRTC
...,...,...,...,...
303,303,Habra to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/habra-to-nan...,WBTC (CTC)
304,304,Habra to Kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...,WBTC (CTC)
305,305,Kolkata to Mayapur ISKCON,https://www.redbus.in/bus-tickets/kolkata-to-m...,WBTC (CTC)
306,306,Habra to Heria,https://www.redbus.in/bus-tickets/habra-to-heria,WBTC (CTC)


## APSRTC bus details - dataframe0 ##

In [99]:
# Collecting APSRTC bus details

driver = webdriver.Chrome() # initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)

# looping through the routes
for i, r in df[df['state'] == 'APSRTC'].iterrows(): # collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2) # waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]') # clicking the route links
    #click the link
    for element in elements: 
        element.click()
        time.sleep(3) #waiting for the page to load
    time.sleep(3)
    try:
        # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)# waiting for the page to load
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source # creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source # creating the scrolled page source 
        if new_page == old_page:# comparing the both pages 
            scrolling = False

        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))

        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit() # quiting the driver


# creating a dataframe to store the datas
dataframe_0 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})




In [102]:
dataframe_0

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:45,05:30,05h 45m,INR 349,4.7\n1045,42 Seats available,Vijayawada to Hyderabad
1,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:25,06:15,06h 50m,INR 329,4.7\n829,36 Seats available,Vijayawada to Hyderabad
2,IntrCity SmartBus,A/C Seater / Sleeper (2+1),22:45,05:45,07h 00m,INR 339,4.5\n735,42 Seats available,Vijayawada to Hyderabad
3,AdSai RK Travels,Non A/C Seater / Sleeper (2+1),22:10,05:30,07h 20m,INR 999,3.8\n128,31 Seats available,Vijayawada to Hyderabad
4,FRESHBUS,Electric A/C Seater (2+2),05:20,12:05,06h 45m,449,4.7\n359,39 Seats available,Vijayawada to Hyderabad
...,...,...,...,...,...,...,...,...,...
2260,NueGo,Electric A/C Seater/Sleeper (2+1),23:30,07:15,07h 45m,664,4.1\n357,32 Seats available,Vijayawada to Visakhapatnam
2261,SRKT Travels,A/C Seater / Sleeper (2+1),23:00,06:30,07h 30m,INR 650,4.3\n193,30 Seats available,Vijayawada to Visakhapatnam
2262,Svkdt travels,A/C Seater / Sleeper (2+1),20:45,04:30,07h 45m,INR 790,4.1\n269,16 Seats available,Vijayawada to Visakhapatnam
2263,Zingbus Plus,Bharat Benz A/C Sleeper (2+1),23:05,07:00,07h 55m,944,4.2\n263,27 Seats available,Vijayawada to Visakhapatnam


## KSRTC (Kerala) Bus details - dataframe1##

In [115]:
# Collecting KSRTC bus details
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'KSRTC (Kerala)'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2) # waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3) #waiting for the page to load
    time.sleep(3)
    try:
        # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)# waiting for the page to load
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages 
            scrolling = False

        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))

        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit() # quiting the driver


# creating a dataframe to store the datas
dataframe_1 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})






In [117]:
dataframe_1

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,23:05,08h 50m,INR 945,4.2\n81,2 Seats available,Bangalore to Kozhikode
1,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,23:45,08h 20m,INR 1099,2.6\n43,1 Seat available,Bangalore to Kozhikode
2,KSRTC (Kerala) - 2851,Super Fast Non AC Seater (2+3),21:48,07:03,09h 15m,INR 688,5.0\n3,13 Seats available,Bangalore to Kozhikode
3,MMK Travels,A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,INR 1199,4.5\n549,10 Seats available,Bangalore to Kozhikode
4,SAAM BUS,VE A/C Sleeper (2+1),21:45,06:45,09h 00m,INR 1450,4.3\n220,13 Seats available,Bangalore to Kozhikode
...,...,...,...,...,...,...,...,...,...
822,Jacobs Travels,Volvo Multi-Axle A/C Semi Sleeper (2+2),18:30,02:30,08h 00m,INR 1299,3.7,27 Seats available,Kottayam to Kozhikode
823,Ubc travels,A/C Semi Sleeper (2+2),17:30,01:00,07h 30m,INR 700,3.7\n3,6 Seats available,Kottayam to Kozhikode
824,Jacobs Travels,A/C Semi Sleeper (2+2),18:30,02:30,08h 00m,INR 800,3.7\n3,9 Seats available,Kottayam to Kozhikode
825,Sonia Travels,A/C Semi Sleeper (2+2),20:10,02:45,06h 35m,INR 700,3.4,22 Seats available,Kottayam to Kozhikode


## TSRTC bus details - dataframe2 ## 

In [120]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'TSRTC'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2) # waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3)#waiting for the page to load
    time.sleep(3)
    try:
        # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)# waiting for the page to load
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages 
            scrolling = False

        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))

        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas
dataframe_2 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})






In [122]:
dataframe_2

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,FRESHBUS,Electric A/C Seater (2+2),04:35,10:30,05h 55m,774,4.7\n436,7 Seats available,Hyderabad to Vijayawada
1,FRESHBUS,Electric A/C Seater (2+2),23:10,05:35,06h 25m,1499,4.5\n489,25 Seats available,Hyderabad to Vijayawada
2,FRESHBUS,Electric A/C Seater (2+2),05:50,11:45,05h 55m,649,4.6\n395,9 Seats available,Hyderabad to Vijayawada
3,AdSai Dasari Travels,Non A/C Seater / Sleeper (2+1),21:30,04:30,07h 00m,INR 990,3.5\n6,10 Seats available,Hyderabad to Vijayawada
4,FRESHBUS,Electric A/C Seater (2+2),11:30,18:05,06h 35m,524,4.6\n299,30 Seats available,Hyderabad to Vijayawada
...,...,...,...,...,...,...,...,...,...
2081,Orange Tours And Travels,NON A/C Seater/ Sleeper (2+1),22:30,06:00,07h 30m,INR 900,3.6\n113,26 Seats available,Hyderabad to Addanki
2082,Sri Vengamamba Bus Transport (SVBT),A/C Seater / Sleeper (2+1),21:40,05:30,07h 50m,INR 1653,4.1\n39,15 Seats available,Hyderabad to Addanki
2083,Chandu Travels,A/C Sleeper (2+1),22:00,05:15,07h 15m,1321,4.3\n28,22 Seats available,Hyderabad to Addanki
2084,Kaveri Travels,Non A/C Seater / Sleeper (2+1),21:00,05:15,08h 15m,INR 1190,4.3\n31,32 Seats available,Hyderabad to Addanki


## Kadamba Transport Corporation Limited (KTCL) bus details - dataframe3 ##

In [7]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'Kadamba Transport Corporation Limited (KTCL)'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2) # waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3)#waiting for the page to load
    time.sleep(3)
    try:
        # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)# waiting for the page to load
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages 
            scrolling = False
            
        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))
        
        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas
dataframe_3 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})



NameError: name 'df' is not defined

In [127]:
dataframe_3

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),10:00,10:31,00h 31m,INR 100,3.7,40 Seats available,Calangute (goa) to Goa
1,Kadamba Transport Corporation Limited (KTCL) -...,Volvo AC Seater 2+2,13:30,14:01,00h 31m,INR 100,1.0\n1,40 Seats available,Calangute (goa) to Goa
2,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,16:01,00h 31m,INR 100,5.0\n1,41 Seats available,Calangute (goa) to Goa
3,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,20:00,11h 30m,INR 400,3.4,17 Seats available,Calangute (goa) to Goa
4,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),10:00,10:31,00h 31m,INR 100,3.7,40 Seats available,Calangute (goa) to Goa
...,...,...,...,...,...,...,...,...,...
63,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,20:00,11h 30m,INR 400,3.4,17 Seats available,Calangute (goa) to Goa
64,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),10:00,10:31,00h 31m,INR 100,3.7,40 Seats available,Calangute (goa) to Goa
65,Kadamba Transport Corporation Limited (KTCL) -...,Volvo AC Seater 2+2,13:30,14:01,00h 31m,INR 100,1.0\n1,40 Seats available,Calangute (goa) to Goa
66,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,16:01,00h 31m,INR 100,5.0\n1,41 Seats available,Calangute (goa) to Goa


## RSRTC bus details - dataframe 4 ## 

In [129]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'RSRTC'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2) # waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3)#waiting for the page to load
    time.sleep(3)
    try:
         # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)# waiting for the page to load
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages 
            scrolling = False

        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))
        
        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas
dataframe_4 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})



In [131]:
dataframe_4

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,RSRTC - 174534,Express Non AC Seater 2+3,05:45,12:00,06h 15m,INR 299,5.0\n1,40 Seats available,Udaipur to Jodhpur
1,RSRTC - 194381,Deluxe AC Seater 2+2,06:00,11:15,05h 15m,INR 527,3.3\n6,4 Seats available,Udaipur to Jodhpur
2,RSRTC - 146395,Express Non AC Seater 2+3,06:45,14:00,07h 15m,INR 299,2.3\n2,42 Seats available,Udaipur to Jodhpur
3,RSRTC - 194422,Deluxe AC Seater 2+2,07:15,12:15,05h 00m,INR 531,4.5\n6,23 Seats available,Udaipur to Jodhpur
4,RSRTC - 194255,Express Non AC Seater 2+3,07:45,14:20,06h 35m,INR 299,4.1,46 Seats available,Udaipur to Jodhpur
...,...,...,...,...,...,...,...,...,...
1113,Vijay Tour and Travels,Bharat Benz A/C Seater / Sleeper (2+2),05:10,09:30,04h 20m,450,4.1\n283,30 Seats available,Jaipur (Rajasthan) to Kota(Rajasthan)
1114,Priya Bus Service,AC Seater (2+2),17:25,22:15,04h 50m,379,3.9\n189,24 Seats available,Jaipur (Rajasthan) to Kota(Rajasthan)
1115,Jakhar Travels,A/C Seater / Sleeper (2+1),22:00,03:30,05h 30m,INR 400,3.9\n98,22 Seats available,Jaipur (Rajasthan) to Kota(Rajasthan)
1116,Babu Travels,A/C Sleeper (2+1),18:30,23:42,05h 12m,INR 600,4.1\n73,26 Seats available,Jaipur (Rajasthan) to Kota(Rajasthan)


## South Bengal State Transport Corporation (SBSTC) bus details - dataframe 5 ##

In [135]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'South Bengal State Transport Corporation (SBSTC)'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2)# waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3)#waiting for the page to load
    time.sleep(3)
    try:
         # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)# waiting for the page to load
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages
            scrolling = False
        
        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))
        
        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas

dataframe_5 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})



In [137]:
dataframe_5

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,WBTC (CTC) Habra - Barasat - Durgapur - Asanso...,Non AC Seater (2+3),06:25,08:00,01h 35m,INR 81,3.1,27 Seats available,Kolkata to Burdwan
1,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),06:00,08:00,02h 00m,INR 429,4.3\n70,14 Seats available,Kolkata to Burdwan
2,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),06:15,08:15,02h 00m,INR 429,4.6\n68,10 Seats available,Kolkata to Burdwan
3,Express Line (Karunamoyee),Volvo A/C Seater (2+2),06:30,08:30,02h 00m,INR 429,4.5\n64,8 Seats available,Kolkata to Burdwan
4,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),07:00,09:15,02h 15m,INR 429,4.6\n131,11 Seats available,Kolkata to Burdwan
...,...,...,...,...,...,...,...,...,...
1931,SBSTC-DURGAPUR - MANBAZAR - 16:10 (MAN BZR) - ...,Non AC Seater (2+3),16:10,17:10,01h 00m,INR 53,3.0,52 Seats available,Durgapur (West Bengal) to Bankura
1932,SBSTC-KARUNAMOYEE - BANKURA - A/C - 15:15 (BNK...,AC Seater (2+2),19:30,00:00,04h 30m,INR 70,4.1\n4,35 Seats available,Durgapur (West Bengal) to Bankura
1933,Tirupati Balaji,NON A/C Seater Push Back (2+3),05:55,07:25,01h 30m,INR 50,1.0\n2,51 Seats available,Durgapur (West Bengal) to Bankura
1934,Dolphin tours and travels,A/C Seater / Sleeper (2+1),17:45,19:00,01h 15m,INR 1008,3.7,3 Seats available,Durgapur (West Bengal) to Bankura


## HRTC bus details - dataframe 6 ## 

In [144]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'HRTC'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2)# waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()#waiting for the page to load
        time.sleep(3)
    time.sleep(3)
    try:
         # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages
            scrolling = False
        
        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))

        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas
dataframe_6 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})



In [146]:
dataframe_6

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,HRTC - 69,Himsuta AC Seater Volvo/Scania 2+2,00:40,09:30,08h 50m,INR 912,4.7\n59,21 Seats available,Delhi to Shimla
1,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,06:45,16:10,09h 25m,INR 912,4.5\n60,28 Seats available,Delhi to Shimla
2,HRTC - 592,A/C Executive (2+3),08:05,18:10,10h 05m,INR 632,3.5\n14,41 Seats available,Delhi to Shimla
3,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,18:40,09h 50m,INR 512,2.6\n4,35 Seats available,Delhi to Shimla
4,HRTC - 7,Himsuta AC Seater Volvo/Scania 2+2,09:25,18:30,09h 05m,INR 912,4.4\n81,20 Seats available,Delhi to Shimla
...,...,...,...,...,...,...,...,...,...
1872,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),16:00,04:20,12h 20m,759,4.7\n659,11 Seats available,Manali to Delhi
1873,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),22:00,07:45,09h 45m,949,4.6\n935,9 Seats available,Manali to Delhi
1874,Laxmi holidays,Volvo 9600 A/C Semi Sleeper (2+2),21:00,07:00,10h 00m,949,4.6\n729,11 Seats available,Manali to Delhi
1875,IntrCity SmartBus,Volvo A/C Semi Sleeper (2+2),19:30,06:45,11h 15m,INR 749,4.7\n377,26 Seats available,Manali to Delhi


## Assam State Transport Corporation (ASTC) bus details - dataframe 7 ##

In [149]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'Assam State Transport Corporation (ASTC)'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2)# waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3)#waiting for the page to load
    time.sleep(3)
    try:
         # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages
            scrolling = False
        
        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))

        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas
dataframe_7 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})



In [151]:
dataframe_7

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,Assam State Transport Corporation (ASTC) - 147480,Bharat Benz A/C Seater (2+2),05:00,09:00,04h 00m,INR 270,3.8\n332,8 Seats available,Tezpur to Guwahati
1,Assam State Transport Corporation (ASTC) - 135927,Volvo AC Seater 2+2,07:10,11:40,04h 30m,INR 298,4.3\n339,25 Seats available,Tezpur to Guwahati
2,Assam State Transport Corporation (ASTC) - 156885,Bharat Benz A/C Seater (2+2),07:35,12:00,04h 25m,INR 298,2.9\n22,33 Seats available,Tezpur to Guwahati
3,Assam State Transport Corporation (ASTC) - 157959,Bharat Benz A/C Seater (2+2),14:00,18:30,04h 30m,INR 298,4.1\n223,24 Seats available,Tezpur to Guwahati
4,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,20:00,04h 15m,INR 298,3.9\n59,25 Seats available,Tezpur to Guwahati
...,...,...,...,...,...,...,...,...,...
487,Orient Transline,A/C Seater (2+1),11:30,15:00,03h 30m,INR 380.95,3.0\n1,37 Seats available,Guwahati to Kaliabor
488,Orient Transline,A/C Seater (2+1),13:00,16:20,03h 20m,INR 380.95,3.0,22 Seats available,Guwahati to Kaliabor
489,BAIKUNTHA TRAVELS (UNDER ASTC),A/C Seater/Sleeper (2+1),20:00,23:15,03h 15m,539,3.1\n3,39 Seats available,Guwahati to Kaliabor
490,Christina Travels,NON A/C Seater (2+2),06:00,09:00,03h 00m,650,2.7,35 Seats available,Guwahati to Kaliabor


## UPSRTC bus details - dataframe 8 ##

In [154]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'UPSRTC'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2)# waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3)#waiting for the page to load
    time.sleep(3)
    try:
         # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages
            scrolling = False

        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))
        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas
dataframe_8 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})



In [156]:
dataframe_8

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,UPSRTC - BLY0112,Janrath AC Seater 2+2,05:31,10:15,04h 44m,INR 598,3.3\n23,35 Seats available,Delhi to Bareilly
1,UPSRTC - STD0161,Ordinary Non AC Seater 2+3,06:00,13:16,07h 16m,INR 448,3.3,52 Seats available,Delhi to Bareilly
2,UPSRTC - GRH0229,Ordinary Non AC Seater 2+3,06:00,12:30,06h 30m,INR 418,3.3,52 Seats available,Delhi to Bareilly
3,UPSRTC - RKD0129,Janrath AC Seater 2+2,06:01,11:01,05h 00m,INR 598,3.2\n16,33 Seats available,Delhi to Bareilly
4,UPSRTC - GRH0230,Ordinary Non AC Seater 2+3,06:30,13:00,06h 30m,INR 418,3.3,52 Seats available,Delhi to Bareilly
...,...,...,...,...,...,...,...,...,...
2144,New Khaira Transport,A/C Seater / Sleeper (2+2),21:20,06:00,08h 40m,INR 500,2.1\n27,34 Seats available,Pilibhit to Delhi
2145,New Khaira TPT,AC Sleeper (2+1),21:30,05:00,07h 30m,INR 900,4.4\n19,30 Seats available,Pilibhit to Delhi
2146,New Khaira Transport,A/C Seater / Sleeper (2+1),22:40,05:00,06h 20m,720,3.4\n33,23 Seats available,Pilibhit to Delhi
2147,Rahul Travels,AshokLeyland Stile A/C,21:45,05:00,07h 15m,INR 826,3.7\n10,26 Seats available,Pilibhit to Delhi


## WBTC (CTC) bus details - dataframe 9 ##

In [159]:
driver = webdriver.Chrome()# initializing the webdriver

# arrays for collecting bus details
bus_name = []
bus_type = []
departure = []
arrival = []
duration = []
price = []
ratings = []
seats_free = []
route_link = []
route_name = []
state_name = []

# initializing wait to improve efficiency
wait = WebDriverWait(driver, 10)
# looping through the routes
for i, r in df[df['state'] == 'WBTC (CTC)'].iterrows():# collecting details
    link = r['links']# collecting the links
    routes = r['routes']# collecting the routes

    driver.get(link)# opening the links in the browser
    time.sleep(2)# waiting for the page to load

    elements =  driver.find_elements(By.XPATH, f'//a[contains(@href, "{link}")]')# clicking the route links
    #click the link
    for element in elements:
        element.click()
        time.sleep(3)#waiting for the page to load
    time.sleep(3)
    try:
         # finding the view bus button for gov bus
        buttons = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button"]')))
        buttons.click()# clicking the button
        time.sleep(2)
    except:
        continue
        
    time.sleep(2)
    # scrolling through the page
    scrolling = True
    while scrolling:
        old_page = driver.page_source# creating an old page source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform# performing page down operaiton usign actionchain
        time.sleep(2)

        new_page = driver.page_source# creating the scrolled page source
        if new_page == old_page:# comparing the both pages
            scrolling = False

        # collecting all datas based on xpath
        ibus_name = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')))
        ibus_type = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')))
        ideparture = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')))
        iarrival = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')))
        iduration = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="dur l-color lh-24"]')))
        try:
            iratings = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]/div[@class="column-six p-right-10 w-10 fl"]')))
        except:
            continue
        iprice = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="fare d-block"]')))
        iseats_free = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "seat-left")]')))

        # storing all datas into the arrays
        for bus in ibus_name:
            bus_name.append(bus.text)
            route_link.append(link)
            route_name.append(routes)
        for type in ibus_type:
            bus_type.append(type.text)
        for start_time in ideparture:
            departure.append(start_time.text)
        for end_time in iarrival:
            arrival.append(end_time.text)
        for total in iduration:
            duration.append(total.text)
        for rating in iratings:
            ratings.append(rating.text)
        for price_elem in iprice:
            price.append(price_elem.text)
        for seats in iseats_free:
            seats_free.append(seats.text)

driver.quit()# quiting the driver

# creating a dataframe to store the datas
dataframe_9 = pd.DataFrame({
    'bus name':bus_name, 
    'bus type':bus_type,
    'departure':departure,
    'arrival':arrival,
    'duration':duration,
    'price':price,
    'ratings':ratings,
    'seats free': seats_free,
    'route name': route_name
})



In [161]:
dataframe_9

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,WBTC (CTC) BARASAT-DIGHA via Bally - 38|03:45,Non AC Seater (2+3),03:45,05:55,02h 10m,INR 90,2.8\n8,28 Seats available,Barasat (West Bengal) to Nandakumar (west bengal)
1,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:30,Non AC Seater (2+3),04:30,06:35,02h 05m,INR 90,4.1\n24,29 Seats available,Barasat (West Bengal) to Nandakumar (west bengal)
2,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:00,Non AC Seater (2+3),05:00,07:05,02h 05m,INR 90,3.5\n18,20 Seats available,Barasat (West Bengal) to Nandakumar (west bengal)
3,WBTC (CTC) Barasat-Digha Via Bally (AC VOLVO) ...,Volvo AC Seater (2+2),05:25,07:30,02h 05m,INR 230,3.4,28 Seats available,Barasat (West Bengal) to Nandakumar (west bengal)
4,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:35,Non AC Seater (2+3),05:35,07:40,02h 05m,INR 90,3.4\n11,26 Seats available,Barasat (West Bengal) to Nandakumar (west bengal)
...,...,...,...,...,...,...,...,...,...
1136,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:30,Non AC Seater (2+3),04:30,08:10,03h 40m,INR 130,4.3\n5,26 Seats available,Habra to Heria
1137,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:15,Non AC Seater (2+3),05:15,08:35,03h 20m,INR 130,4.2\n9,33 Seats available,Habra to Heria
1138,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:35,Non AC Seater (2+3),05:35,08:55,03h 20m,INR 130,4.7\n4,26 Seats available,Habra to Heria
1139,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:55,Non AC Seater (2+3),06:55,10:45,03h 50m,INR 130,3.0\n1,38 Seats available,Habra to Heria


In [163]:
# creating a result database by joining all databases
result_bus_detail = pd.concat([dataframe_0, dataframe_1, dataframe_2, dataframe_3, dataframe_4, dataframe_5, dataframe_6, dataframe_7, dataframe_8, dataframe_9], ignore_index=True)

result_bus_detail

,bus name,bus type,departure,arrival,duration,price,ratings,seats free,route name
0,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:45,05:30,05h 45m,INR 349,4.7\n1045,42 Seats available,Vijayawada to Hyderabad
1,IntrCity SmartBus,A/C Seater / Sleeper (2+1),23:25,06:15,06h 50m,INR 329,4.7\n829,36 Seats available,Vijayawada to Hyderabad
2,IntrCity SmartBus,A/C Seater / Sleeper (2+1),22:45,05:45,07h 00m,INR 339,4.5\n735,42 Seats available,Vijayawada to Hyderabad
3,AdSai RK Travels,Non A/C Seater / Sleeper (2+1),22:10,05:30,07h 20m,INR 999,3.8\n128,31 Seats available,Vijayawada to Hyderabad
4,FRESHBUS,Electric A/C Seater (2+2),05:20,12:05,06h 45m,449,4.7\n359,39 Seats available,Vijayawada to Hyderabad
...,...,...,...,...,...,...,...,...,...
13954,WBTC (CTC) HABRA-DIGHA via Bally - 26|04:30,Non AC Seater (2+3),04:30,08:10,03h 40m,INR 130,4.3\n5,26 Seats available,Habra to Heria
13955,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:15,Non AC Seater (2+3),05:15,08:35,03h 20m,INR 130,4.2\n9,33 Seats available,Habra to Heria
13956,WBTC (CTC) HABRA-DIGHA via Bally - 26|05:35,Non AC Seater (2+3),05:35,08:55,03h 20m,INR 130,4.7\n4,26 Seats available,Habra to Heria
13957,WBTC (CTC) HABRA-DIGHA via Bally - 26|06:55,Non AC Seater (2+3),06:55,10:45,03h 50m,INR 130,3.0\n1,38 Seats available,Habra to Heria


In [167]:
#converting dataframe to csv files
result_bus_detail.to_csv('bus_details_complete.csv')